In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()
driver.get("http://image-net.org/")

In [5]:
elem = driver.find_element_by_name('q')
elem.clear()
elem.send_keys('dog')
elem.send_keys(Keys.RETURN)

In [13]:
source = driver.find_element_by_class_name('search_result')
source = source.get_attribute('innerHTML')

In [40]:
import re

res = re.findall('(synset\?wnid=n[\d]{8})', source)

unique_wnids = set()

for match in res:
    re1 = re.search('n[\d]{8}', match)
    unique_wnids.add(re1.group())

print(unique_wnids)

{'n02111277', 'n02106662', 'n02109961', 'n03218198', 'n02110341', 'n02116738'}


In [48]:
import requests

unique_wnids = list(unique_wnids)
uri_prefix = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid='

train_urls, test_urls = [], []

for wnid in unique_wnids:
    page = requests.get(f'{uri_prefix}{wnid}')
    uri_list = page.text.split('\r\n')

    split_index = int(len(uri_list) * 0.8)

    train_urls.extend(uri_list[:split_index])
    test_urls.extend(uri_list[split_index:])


len(train_urls), len(test_urls)

(6983, 1749)

In [50]:
train_urls = list(set(train_urls))
test_urls = list(set(test_urls))

In [74]:
import urllib
import cv2
import numpy as np

def get_image_name(url):
    return url.rsplit('/', 1)[-1]

In [73]:
from skimage import io
import os 

def store_in_foler(folder_name, urls, max_files = -1):
    stored = 0
    for url in urls:
        if stored <= max_files or max_files == -1:
            try:
                img = io.imread(url)
                cv2.imwrite(f'./{folder_name}/{get_image_name(url)}', cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                stored = stored + 1
            except Exception:
                pass